# The goal of this tutorial is use simple post processing on a rst file and export it in vtk for visualization

To use it, add PYTHONPATH to the generated python code from Protocols/packages and ANSYS_PATH to your ansys install path

1st step : load the necessary module of dpf for grpc, launch the server and create the doc

In [1]:
from ansys import dpf
from ansys.dpf.server import start_local_server

In [2]:
start_local_server()

server started at --address 10.110.2.47 --port 50052


In [3]:
doc = dpf.Operator("html_doc")
doc.inputs.output_path(r'c:/temp/doc.html')
doc.eval()

2nd step : create a model with the path of the result file

In [4]:
path = r'c:/temp/cp55.rst'
model = dpf.Model(path)

3rd step : get the time freq support

In [5]:
print(model.time_freq_support)

Time/Frequency Info:
	Number of sets: 35

With complex values
 
 Cumulative      Time (s)       Loadstep     Substep   
     1             0.0             1            1      
     2             0.02            1            2      
     3             0.04            1            3      
     4             0.06            1            4      
     5             0.08            1            5      
     6             0.1             1            6      
     7             0.12            1            7      
     8             0.14            1            8      
     9             0.16            1            9      
     10            0.18            1            10     
     11            0.2             1            11     
     12           0.219            1            12     
     13           0.239            1            13     
     14           0.259            1            14     
     15           0.279            1            15     
     16           0.299            1    

4th step: get the mesh

In [6]:
print(model.meshed_region)

Meshed Region
	3820 nodes
	789 elements
	Unit: m 



5th step: look for available results in the file

In [7]:
print(model.result_info)

Static analysis
Result types: 15 
Unit system: 11
Physics Type: Mecanic
Available results:
	U
	RF
	ENF
	S
	ENG_VOL
	ENG_SE
	ENG_AHO
	ENG_TH
	ENG_KE
	ENG_CO
	ENG_INC
	EPEL
	ETH_EQV
	ETH_SWL
	BFE



## Using displacement

4 th step : create the operator reader of displacement and set it's time scoping request as the entire time freq support
connect it to a norm and to a min max and export the norm in vtk

In [8]:
displacements = model.operator("U")
timeids = list(range(1,model.time_freq_support.n_sets+1))

In [9]:
displacements.inputs.time_scoping(timeids)

In [10]:
norm = dpf.Operator("norm_fc")
norm.inputs.fields_container.connect(displacements.outputs)

In [11]:
min_max = dpf.Operator("min_max_fc")
min_max.inputs.fields_container.connect(norm.outputs)
fmin = min_max.outputs.field_min()
fmax = min_max.outputs.field_max()

In [12]:
fmin.data

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [13]:
fmax.data

[0.0,
 0.0006267372903018056,
 0.0025094009142665004,
 0.005641846246302676,
 0.009999923398266329,
 0.015521540117110409,
 0.022078712720450363,
 0.029444587009846576,
 0.03725894252314807,
 0.0449972152013357,
 0.05195353454479884,
 0.05703911801494583,
 0.05982844150784884,
 0.0589761684373874,
 0.05358419092854356,
 0.04310435808459619,
 0.027597819239314075,
 0.007984309803602908,
 0.01379509993808897,
 0.03478255103903859,
 0.05130461351335084,
 0.05942391889982178,
 0.05715204142158222,
 0.04272116079422172,
 0.01787116097063537,
 0.012449936699550468,
 0.04062977331496594,
 0.05913065884847142,
 0.060420564719179184,
 0.04188290338330022,
 0.012018786803938152,
 0.0352653150526427,
 0.05950851527080637,
 0.06077103487616713,
 0.037337691401448825]

In [14]:
vtk = dpf.Operator("vtk_export")
vtk.inputs.mesh(model.meshed_region)
vtk.inputs.fields(norm.outputs)
vtk.inputs.file_path(r'c:/temp/file.vtk')
vtk.eval()